#### Required imports for analysis part

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# pdb2gmx takes only protein file containing standard amino acid residues,
    # so check the protein file, should contain only standard residues
! grep -v HETATM protein.pdb > prot_clean.pdb

### Generate Topology

In [ ]:
# add 3point water (TIP or SPC) model unlesss 4point water is necessary
! gmx pdb2gmx -f prot_clean.pdb -o prot_pros.gro -water spce -ff amber99sb

In [ ]:
! tail topol.top

### create box and add water in box

In [ ]:
# c- center the molecule in box, d-distance from box edge to molecule, bt-box type
! gmx editconf -f prot_pros.gro -o prot_box.gro -c -d 1.0 -bt dodecahedron

In [ ]:
#cp-protein with box vectors, cs-solvent gro file, o-solvated system, p-topology file
! gmx solvate -cp prot_box.gro -cs spc216.gro -o prot_solv.gro -p topol.top

### Add ions

In [ ]:
! gmx grompp -f ions.mdp -c prot_solv.gro -p topol.top -o ions.tpr

In [ ]:
! printf "SOL" | gmx genion -s ions.tpr -o prot_solv_ions.gro -p topol.top -pname NA -nname CL -neutral

In [ ]:
! tail topol.top

### Energy minimization and potential energy graph

In [ ]:
# review nsteps
! gmx grompp -f minim.mdp -c prot_solv_ions.gro -p topol.top -o em.tpr

In [ ]:
# ! gmx mdrun -v -deffnm em -nt 8 &
# ampersand used to continue running even the shell is closed in linux shells in general
! gmx mdrun -v -deffnm em -nt 8

In [ ]:
! printf "10" | gmx energy -f em.edr -o potential.xvg

In [ ]:
potential = np.genfromtxt([i for i in open('potential.xvg').read().splitlines() 
    if not i.startswith(('#','@'))])

plt.plot(*potential.T)
plt.xlabel('stop')
plt.ylabel('potential')

### Equlibration steps NVT

In [ ]:
# Enter number of steps and multiply by *2
# after multplying now 1step = 1fs
# fs -> ps -> ns

# review nsteps and output control in mdp file
! gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

In [ ]:
! gmx mdrun -v -deffnm nvt -nt 8

In [ ]:
# ! gmx mdrun -v -deffnm nvt -cpi nvt.cpt -nt 6
# if md run is interruprted then uncomment and excecute this cell to continue operation 

In [ ]:
! printf "15" | gmx energy -f nvt.edr -o temperature.xvg 

In [ ]:
temperature = np.genfromtxt([i for i in open('temperature.xvg').read().splitlines() 
    if not i.startswith(('#','@'))])  

plt.plot(*temperature.T)
plt.xlabel('stop')
plt.ylabel('temperature')

### Equlibration NPT

In [ ]:
# review nsteps and output control in mdp file
! gmx grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr 

In [ ]:
! gmx mdrun -v -deffnm npt -nt 8

In [ ]:
# ! gmx mdrun -v -deffnm npt -cpi npt.cpt -nt 6
# if md run is interruprted then uncomment and excecute this cell to continue operation 

In [ ]:
! printf "17" | gmx energy -f npt.edr -o pressure.xvg 

In [ ]:
pressure = np.genfromtxt([i for i in open('pressure.xvg').read().splitlines() 
    if not i.startswith(('#','@'))])

plt.plot(*pressure.T)
plt.xlabel('stop')
plt.ylabel('pressure')

In [ ]:
! printf "24" | gmx energy -f npt.edr -o density.xvg 

In [ ]:
density = np.genfromtxt([i for i in open('density.xvg').read().splitlines() 
    if not i.startswith(('#','@'))]) 

plt.plot(*density.T)
plt.xlabel('stop')
plt.ylabel('density')

### Equlibration molecular dynamics - production MD run

In [ ]:
# review nsteps and output control in mdp file
! gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr

In [ ]:
! gmx mdrun -v -deffnm md -nt 8

In [ ]:
# ! gmx mdrun -v -deffnm md -cpi md.cpt -nt 8

### Trajectory correction

In [ ]:
# correct production md run trajectory that only needed
! printf "Protein\nSystem\n" | gmx trjconv -s md.tpr -f md.xtc -center -ur compact -pbc mol -o md_corrected.xtc

#### Trajectory visualisation from previous file

In [ ]:
# to visualise the trajectories in vmd
# vmd md.gro md_correccted.xtc

### Analysis

In [ ]:
# 1. RMSD
! printf "Protein\nProtein\n" | gmx rms -s md.tpr -f md_corrected.xtc -o rmsd.xvg

In [ ]:
rmsd = np.genfromtxt([i for i in open('rmsd.xvg').read().splitlines() 
    if not i.startswith(('#','@'))]) 

plt.plot(*rmsd.T)
plt.xlabel('steps')
plt.ylabel('rmsd')